### Graded Lab 5

Hello ! Welcome to Graded Lab of Module 5.

In the last assignment we had worked on missing values & outliers.
Its high time when we look at the feature engineering & Principal component analysis.

**So lets go ahead with the last graded assignment for the course !**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Reading sales data
sales = pd.read_csv('sales_data.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code
0,1,MX-2014-143658,02/10/2014,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium,VE_001
1,2,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium,VE_002
2,3,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium,VE_003
3,4,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium,VE_004
4,5,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium,VE_005


In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [2]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    
    data = data1.merge(right=data2, how="inner", on="customer_id")
    data = data.merge(right=data3, how="inner", on="vendor_code")
    data["order_date"] = pd.to_datetime(data['order_date'], format='%d/%m/%Y')
    return data

In [3]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [4]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

In [5]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'vendor_code', 'customer_name', 'city', 'state',
       'country', 'postal_code', 'segment', 'market', 'region', 'vendor'],
      dtype='object')

### Q.1) Create a feature for identifying which orders are delivered on weekends. Flag will take value 1 if order is delivered on weekends & 0 otherwise. (Sat & Sun are weekeends). Return  the output series.

In [6]:
def weekend_flag(data):
    
    """
  Creates a feature identifying weekend orders and returns the flag series.

  Args:
    data (pd.DataFrame): The DataFrame containing the order data.

  Returns:
    pd.Series: A Series with 1 for weekend orders and 0 for others.
    """
    data['weekend_orders'] = np.where(data['order_date'].dt.weekday > 4, 1, 0)  # Assign 1 for weekends, 0 otherwise
    
    return data['weekend_orders']  # Return the flag series

In [7]:
sales['weekend_orders']=weekend_flag(data=sales)

In [8]:
sales['weekend_orders'].value_counts()  #PASSED

0    45720
1     5570
Name: weekend_orders, dtype: int64

In [9]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert all(sales['weekend_orders'].unique()==np.array([0, 1])) ,'Make sure that you have created flag correctly'


### Q.2) We wish to create a feature called 'discount_per_quantity'. Write a functional code to compute ratio of 2 columns. Return  the output series.

In [10]:
def feature_1(data,col1,col2):
    """
  Computes the ratio of two columns and returns the result as a new feature series.

  Args:
    data (pd.DataFrame): The DataFrame containing the data.
    col1 (str): Name of the first column for the ratio.
    col2 (str): Name of the second column for the ratio.

  Returns:
    pd.Series: A Series containing the ratio of col1 to col2 for each row.
    """
    data['discount_per_quantity'] = data[col1] / data[col2]  # Calculate the ratio
    
    return data['discount_per_quantity']

In [11]:
sales['discount_per_quantity']= feature_1(data=sales,col1='discount',col2='quantity')

In [12]:
sales['discount_per_quantity'].value_counts()

0.000000    29009
0.100000     2291
0.050000     1656
0.066667     1471
0.200000     1376
            ...  
0.027273        1
0.053846        1
0.013077        1
0.011538        1
0.100333        1
Name: discount_per_quantity, Length: 219, dtype: int64

In [13]:
round(sales['discount_per_quantity'].mean(),4)  #PASSED

0.0643

In [14]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert round(sales['discount_per_quantity'].mean(),4)==0.0643,'Make sure that you have created feature correctly.'


### Q.3) Create a feature for finding cumulative sales for every customer. Return  the output series.

In [15]:
def cumulative_sales(data):
    
    """
  Creates a feature for cumulative sales for each customer and returns the series.

  Args:
    data (pd.DataFrame): The DataFrame containing sales data with a 'customer_id' column.

  Returns:
    pd.Series: A Series containing cumulative sales for each customer.
    """
    
    data['cumulative_sales'] = data.groupby('customer_id')['sales'].transform(pd.Series.cumsum)  # Calculate cumulative sales
    
    return data['cumulative_sales']  # Return the new feature series

In [16]:
### Sorting the dataframe wrto customer_id & order_date
sales.sort_values(['customer_id','order_date'],ascending=[1,1],inplace=True)
### Applying cumulative_sales function
sales['cumulative_sales']=cumulative_sales(data=sales)

In [17]:
sales['cumulative_sales']

29198     673.5680
29199     726.5480
8548      778.2590
8549      823.1885
8550     1102.0385
           ...    
46464    8100.9629
47330    8546.2889
16678    8762.2889
18475    9350.3114
49055    9479.3444
Name: cumulative_sales, Length: 51290, dtype: float64

In [18]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert round(sales[sales['customer_id']=='ZC-21910']['cumulative_sales'].max(),2)==28472.82

In [19]:
round(sales[sales['customer_id']=='ZC-21910']['cumulative_sales'].max(),2) #PASSED

28472.82

### Q.4) Please demonstrate how to enhance a given dataset by introducing a new feature that records the frequency of same-day purchases made by each customer. The desired outcome should be a modified dataframe containing a variable named 'same_day_purchase_frequency' which accurately portrays the same-day purchase behavior of individual customers.
#### same_day_purchase_frequency will be the frequency of the orders made on same day. If customer 'ABC' has placed 2 orders on 2nd Jan 23 then 'same_day_purchase_frequency'for this customer for 2nd Jan 23 will be 2.  

In [20]:
#sales.columns

In [21]:
# TROUBLESHOOTING FUNCTION

# def same_day_purchase(data):
    
#     # Group by 'customer_id' and date, and count the orders, store count of orders in variable named 'order_count'
#     order_counts = data.groupby(['customer_id', 'order_date'])['order_id'].count()  # Assuming 'order_id' is unique per order
    
#     # Filter for same-day purchases (order_count > 1)
#     same_day_purchases = order_counts[order_counts > 1]
    
#     # Group by 'customer_id' and calculate the same-day purchase frequency , name that column as 'same_day_purchase_frequency'
#     same_day_purchase_frequency = same_day_purchases.groupby('customer_id').size().rename('same_day_purchase_frequency')
    
#     # Merge the result with the original DataFrame to add the same-day purchase frequency as a new column  
#     data = data.merge(same_day_purchase_frequency.to_frame().reset_index(), on='customer_id', how='left')
    
#     # Fill missing values with 0 (for customers with no same-day purchases)
#     data['same_day_purchase_frequency'].fillna(0, inplace=True)
    
#     return data


In [22]:
#same_day_purchase(sales.copy())

In [23]:
#same_day_purchase(sales.copy()).isnull().sum()

In [24]:
def same_day_purchase(data):
    
    # Group by 'customer_id' and date, and count the orders, store count of orders in variable named 'order_count'
    order_counts = data.groupby(['customer_id', 'order_date'])['order_id'].count()  # Assuming 'order_id' is unique per order
    
    # Filter for same-day purchases (order_count > 1)
    same_day_purchases = order_counts[order_counts > 1]
    
    # Group by 'customer_id' and calculate the same-day purchase frequency , name that column as 'same_day_purchase_frequency'
    same_day_purchase_frequency = same_day_purchases.groupby('customer_id').size().rename('same_day_purchase_frequency')
    
    # Merge the result with the original DataFrame to add the same-day purchase frequency as a new column  
    data = data.merge(same_day_purchase_frequency.to_frame().reset_index(), on='customer_id', how='left')
    
    # Fill missing values with 0 (for customers with no same-day purchases)
    data['same_day_purchase_frequency'].fillna(0, inplace=True)
    
    return data

In [25]:
sales_new=same_day_purchase(data=sales.copy())
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert float(sales_new[sales_new['customer_id']=='ZC-11910']['same_day_purchase_frequency'].unique())==0,'Make sure that you have created variable correctly.'


In [26]:
sales_new = same_day_purchase(data=sales.copy())
sales_new.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,...,country,postal_code,segment,market,region,vendor,weekend_orders,discount_per_quantity,cumulative_sales,same_day_purchase_frequency
0,33526,CA-2011-128055,2011-03-31,05/04/2011,Standard Class,AA-10315,OFF-BI-10004390,Office Supplies,Binders,GBC DocuBind 200 Manual Binding Machine,...,Mexico,NaN,Consumer,LATAM,LATAM-North,Lifoods,0,0.100000,673.5680,10.0
1,33527,CA-2011-128055,2011-03-31,05/04/2011,Standard Class,AA-10315,OFF-AP-10002765,Office Supplies,Appliances,Fellowes Advanced Computer Series Surge Protec...,...,Mexico,NaN,Consumer,LATAM,LATAM-North,Lifoods,0,0.000000,726.5480,10.0
2,27384,ID-2011-40953,2011-07-04,07/07/2011,First Class,AA-10315,OFF-SU-10003723,Office Supplies,Supplies,"Elite Shears, High Speed",...,Mexico,NaN,Consumer,LATAM,LATAM-North,Low Tide Corp,0,0.225000,778.2590,10.0
3,27385,ID-2011-40953,2011-07-04,07/07/2011,First Class,AA-10315,OFF-LA-10000425,Office Supplies,Labels,"Avery Shipping Labels, Alphabetical",...,Mexico,NaN,Consumer,LATAM,LATAM-North,Low Tide Corp,0,0.064286,823.1885,10.0
4,27387,ID-2011-40953,2011-07-04,07/07/2011,First Class,AA-10315,OFF-SU-10001308,Office Supplies,Supplies,"Fiskars Trimmer, Serrated",...,Mexico,NaN,Consumer,LATAM,LATAM-North,Low Tide Corp,0,0.034615,1102.0385,10.0


In [27]:
float(sales_new[sales_new['customer_id']=='ZC-11910']['same_day_purchase_frequency'].unique()) #PASSED

0.0

### Q.5) Create a variable which will calculate weighted sales per market. Return  the output series.
Weights for every region are :-'LATAM':1, 'EMEA':1.3, 'Africa':2.7, 'Canada':1.4, 'EU':2.8, 'APAC':3.5

In [28]:
sales.market.value_counts()

LATAM     41265
EMEA       4884
Africa     4798
Canada      313
EU           23
APAC          7
Name: market, dtype: int64

In [29]:
def weighted_sales(data):
    # Write a lambda function & apply it to dataframe.Kindly ignore raiseNotImplementError
    
    """
    Calculates weighted sales per market and returns the output series.

    Args:
        data (pd.DataFrame): The DataFrame containing sales data with 'market' and 'sales' columns.

    Returns:
        pd.Series: A Series containing weighted sales for each market.
    """
    
    weights = {'LATAM': 1, 'EMEA': 1.3, 'Africa': 2.7, 'Canada': 1.4, 'EU': 2.8, 'APAC': 3.5}
    
    # Define a lambda function to calculate weighted sales
    calculate_weighted_sales = lambda row: row['sales'] * weights[row['market']]
    
    # Apply the lambda function to the DataFrame to create the new feature
    data['weighted_market_sales'] = data.apply(calculate_weighted_sales, axis=1)
    
    return data['weighted_market_sales']  # Return the new feature series 

In [30]:
sales['weighted_market_sales']=weighted_sales(data=sales)

In [31]:
sales['weighted_market_sales'].head()

29198    673.5680
29199     52.9800
8548      51.7110
8549      44.9295
8550     278.8500
Name: weighted_market_sales, dtype: float64

In [32]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert round(float(sales[(sales['market']=='Africa') & (round(sales['sales'],4)==58.83)]['weighted_market_sales'].unique()),2)==158.84, 'Make sure that ypu have created column correctly.'

In [33]:
round(float(sales[(sales['market']=='Africa') & (round(sales['sales'],4)==58.83)]['weighted_market_sales'].unique()),2)#PASSED

158.84

Most of the variables in our data are categorical in nature & for further anlysis purpose if we convert them by ordinal encoding or one hot encoding, we will be left with too many variables.
If we have too many variables in the data , it might cause trouble in later stages like machine learning model building.
1. Having too many variables can lead to overfitting and poor generalization to new data.
2. It can make it difficult to analyze and interpret the data and identify the most important variables.
3. It can increase computational complexity and time required to build and train a model.

How to deal with this ?

Wait , we have learned about **Principal component analysis** , we can leverage that method here for **dimensionality reduction.**

### Q.6 ) Complete the following code for PCA implementation. Return the no of PC's required for capturing the 80% of variation, variance explained by the PC's .

In [34]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'vendor_code', 'customer_name', 'city', 'state',
       'country', 'postal_code', 'segment', 'market', 'region', 'vendor',
       'weekend_orders', 'discount_per_quantity', 'cumulative_sales',
       'weighted_market_sales'],
      dtype='object')

In [35]:
#sales["sub_category"]

In [36]:
### TROUBLESHOOT FUNCTION

# def PCA(data,target=0.80):
    
    
#     # Treating ordinal variable ,
#     # order_priority : 'Medium':2, 'Low':1, 'High':3, 'Critical':4 
#     data['order_priority_encoded'] = data['order_priority'].map({'Medium': 2, 'Low': 1, 'High': 3, 'Critical': 4})
    
#     # Treating missing values
#     # Impute the missing values in 'order_priority_encoded','ship_mode' by mode, 
#     data['order_priority_encoded'].fillna(data['order_priority_encoded'].mode()[0], inplace=True)
#     data['ship_mode'].fillna(data['ship_mode'].mode()[0], inplace=True)
    
#     # Treating Nominal variables ,'ship_mode','category','sub_category' 
#     # (while encoding them make sure to select k-1 dummy variables for a variable with k categories)
#     data = pd.get_dummies(data, columns=['ship_mode', 'category', 'sub_category'], drop_first=True)
    
#     # Final data selection
#     # Select all the variables with 'sales','quantity','discount','profit','shipping_cost',
#     # order_priority_encoded' & dummies in 'ship_mode','category','sub_category'.
    
#     data_1 = data[['sales','quantity','discount','profit','shipping_cost','order_priority_encoded',
#                    'ship_mode_Same Day','ship_mode_Second Class','ship_mode_Standard Class',
#                    'category_Office Supplies','category_Technology','sub_category_Appliances',
#                    'sub_category_Art','sub_category_Binders','sub_category_Bookcases','sub_category_Chairs',
#                    'sub_category_Copiers','sub_category_Envelopes','sub_category_Fasteners',
#                    'sub_category_Furnishings','sub_category_Labels','sub_category_Machines',
#                    'sub_category_Paper','sub_category_Phones','sub_category_Storage',
#                    'sub_category_Supplies','sub_category_Tables']]
    
#     # Standardize the data
#     sc = StandardScaler()
    
#     cols = ['sales','quantity','discount','profit','shipping_cost','order_priority_encoded']
    
#     for col in cols:
#         data_1[col] = sc.fit_transform(data_1[[col]])
        
#     from sklearn.decomposition import PCA
    
#     # Create a PCA instance    
#     pca = PCA()
    
#     # Fit PCA to the scaled data
#     pca.fit(data_1) # Write your code in place of None
    
#     # Calculate cumulative explained variance
#     explained_variance_ratio = pca.explained_variance_ratio_
#     cumulative_variance = np.cumsum(explained_variance_ratio)

#     target_variation = target
#     n_components = np.argmax(cumulative_variance >= target_variation) + 1
    
#     # Excellent we got no of pcs which are required for capturing the target variation in the data
    
#     ### Fitting PCA again with no of pc's we got above.
#     pca = PCA(n_components=n_components)
    
#     # Fit PCA to the scaled data
#     pca.fit(data_1)
    
#     #  Calculate cumulative explained variance
#     explained_variance_ratio = pca.explained_variance_ratio_
       
#     return n_components,explained_variance_ratio    

In [37]:
#PCA(sales.copy(), target=0.80)

In [38]:
def PCA(data,target=0.80):
    
    
    # Treating ordinal variable ,
    # order_priority : 'Medium':2, 'Low':1, 'High':3, 'Critical':4 
    data['order_priority_encoded'] = data['order_priority'].map({'Medium': 2, 'Low': 1, 'High': 3, 'Critical': 4})
    
    # Treating missing values
    # Impute the missing values in 'order_priority_encoded','ship_mode' by mode, 
    data['order_priority_encoded'].fillna(data['order_priority_encoded'].mode()[0], inplace=True)
    data['ship_mode'].fillna(data['ship_mode'].mode()[0], inplace=True)
    
    # Treating Nominal variables ,'ship_mode','category','sub_category' 
    # (while encoding them make sure to select k-1 dummy variables for a variable with k categories)
    data = pd.get_dummies(data, columns=['ship_mode', 'category', 'sub_category'], drop_first=True)
    
    # Final data selection
    # Select all the variables with 'sales','quantity','discount','profit','shipping_cost',
    # order_priority_encoded' & dummies in 'ship_mode','category','sub_category'.
    
    data_1 = data[['sales','quantity','discount','profit','shipping_cost','order_priority_encoded',
                   'ship_mode_Same Day','ship_mode_Second Class','ship_mode_Standard Class',
                   'category_Office Supplies','category_Technology','sub_category_Appliances',
                   'sub_category_Art','sub_category_Binders','sub_category_Bookcases','sub_category_Chairs',
                   'sub_category_Copiers','sub_category_Envelopes','sub_category_Fasteners',
                   'sub_category_Furnishings','sub_category_Labels','sub_category_Machines',
                   'sub_category_Paper','sub_category_Phones','sub_category_Storage',
                   'sub_category_Supplies','sub_category_Tables']]
    
    # Standardize the data
    sc = StandardScaler()
    
    cols = ['sales','quantity','discount','profit','shipping_cost','order_priority_encoded']
    
    for col in cols:
        data_1[col] = sc.fit_transform(data_1[[col]])
        
    from sklearn.decomposition import PCA
    
    # Create a PCA instance    
    pca = PCA()
    
    # Fit PCA to the scaled data
    pca.fit(data_1) # Write your code in place of None
    
    # Calculate cumulative explained variance
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance_ratio)

    target_variation = target
    n_components = np.argmax(cumulative_variance >= target_variation) + 1
    
    # Excellent we got no of pcs which are required for capturing the target variation in the data
    
    ### Fitting PCA again with no of pc's we got above.
    pca = PCA(n_components=n_components)
    
    # Fit PCA to the scaled data
    pca.fit(data_1)
    
    #  Calculate cumulative explained variance
    explained_variance_ratio = pca.explained_variance_ratio_
       
    return n_components,explained_variance_ratio

In [39]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert type(PCA(data=sales.copy())[0])==np.int64 , 'Number of components required for capturing target variation should be in integer format.'
assert type(PCA(data=sales.copy())[1])==np.ndarray , 'Type of variation captured should be in an array format.'
assert [round(x,4) for x in list(PCA(data=sales.copy())[1])][0]==0.3012, 'Make sure that you are returning correct values for output.'


In [40]:
PCA(data=sales.copy())[0]

6

In [41]:
type(PCA(data=sales.copy())[0])

numpy.int64

In [42]:
PCA(data=sales.copy())[1]

array([0.30122233, 0.14869927, 0.1394773 , 0.1093968 , 0.07239173,
       0.04129315])

In [43]:
type(PCA(data=sales.copy())[1])

numpy.ndarray

In [44]:
list(PCA(data=sales.copy())[1])[0]

0.30122233079178184

In [45]:
[round(x,4) for x in list(PCA(data=sales.copy())[1])][0]  #PASSED

0.3012

### Q.7) How much variation was explained by 3rd principal component ? 
**Round output rounded upto 4 digits**

In [46]:
def var_expl(variation_explained):
       
    """
    Extracts and rounds the explained variance of the 3rd principal component.

    Args:
        variation_explained (list or array): A list or array containing the explained variance of each principal component.

    Returns:
        float: The explained variance of the 3rd principal component, rounded to 4 decimal places.
    """

    third_pc_variance = variation_explained[2]  # Access the 3rd PC's explained variance (index 2)
    rounded_variance = round(third_pc_variance, 4)  # Round to 4 decimal places
    
    return rounded_variance
    #PASSED

In [47]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.